# The Message Passing Interface MPI
## Introduction  to MPI
The Message Passing Interface (MPI) is:

- Particularly useful for distributed memory machines
- The _de facto_ standard parallel programming interface

Many implementations exist - MPICH, OpenMPI, ...

Interfaces in 
- C/C++ 
- Fortran and ... 
- Python wrappers (MPI4Py)

## Message Passing Paradigm
The parallel program is launched as separate processes (tasks) each with their own address space.
- To achieve parallelism we should partition data across tasks 

Data must be **explicitly moved** from process to process:
- A task can access the data of another process through passing a message (a copy of the data is passed from one process to another)

Two main classes of message passing:
- **Point-to-point** operations, involving only two processes
- **Collective** operations, involving a group of processes

## MPI4Py
MPI4Py provides an interface similar to the MPI standard C++ interface

You can communicate Python objects
 - e.g. entire numpy arrays, rather than splitting into data and metadata that surrounds the object
    

## Communicators
MPI uses communicator objects to identify a set of processes that can communicate with each other

- `MPI_COMM_WORLD` is a default communicator, which contains all processes

Processes have ranks 
- Unique process id in a given communicator, assigned by the system when the process initializes
- Used to specify the sources and destinations of messages
- 0 is the "root process", often used for I/O, etc.

### Hello World!

In [ ]:
%%file hello_world_mpi.py
#import the MPI class from mpi4py
from mpi4py import MPI
# call the COMM_WORLD attribute, store that in comm
comm = MPI.COMM_WORLD
# one of the attributes comm has is rank
print("Hello world, I am process: " + str(comm.rank))

In [ ]:
!mpirun -n 2 --mca mpi_cuda_support 0 python hello_world_mpi.py

## Point to Point Communication

Point-to-point communication is sending message/data from one process to another. 



In [ ]:
%%file send_and_receive.py
from mpi4py import MPI
import numpy
comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size
a = numpy.array([rank]*10, dtype=float)
if rank == 0:
    comm.send(a, dest = (rank + 1) % size)
if rank > 0:
    data = comm.recv(source = (rank - 1) % size)
    comm.send(a, dest = (rank + 1) % size)
if rank == 0:
    data = comm.recv(source = size - 1)
print("My ranks is " + str(rank) + "\n and I received this array:\n" + str(data))

In [ ]:
!mpirun -n 2 --mca mpi_cuda_support 0 python send_and_receive.py

## Collective communication 
Generally groups of processes need to exchange messages between themselves. Rather than explicitly sending and receiving such messages from point to point, MPI comes with group operations known as collectives.
- Broadcast, scatter, gather and reduction
- Implementations can optimize performance

### Broadcast
Send from one process to all other processes.

### Scatter
Split data into chunks and send a chunk to individual processes to work on.

### Gather
Gather the chunks and bring them to the root process

### Reduction
Gather, and do some computation.

## Scatter
We create an array on rank 0 and scatter it to all ranks.

In [ ]:
%%file scatter.py
from mpi4py import MPI
import numpy
comm = MPI.COMM_WORLD
sendbuf = []
if comm.rank == 0:
    m = numpy.random.randn(comm.size, comm.size)
    print("Original array on root process\n" + str(m))
    sendbuf = m
# call this on every rank, including rank 0
v = comm.scatter(sendbuf, root=0)
print("I got this data:\n" + str(v) + "\n and my ranks is " + str(comm.rank))

In [ ]:
!mpirun -n 2 --mca mpi_cuda_support 0 python scatter.py

## Gather
Collect the results from all processes onto rank 0.

In [ ]:
%%file gather.py
from mpi4py import MPI
import numpy
comm = MPI.COMM_WORLD
sendbuf = []

if comm.rank == 0:
    m = numpy.array(range(comm.size * comm.size), dtype=float)
    m.shape=(comm.size, comm.size)
    print("Original array on root process\n" + str(m))
    sendbuf = m
    
# first scatter like before
v = comm.scatter(sendbuf, root=0)
print("I got this data:\n" + str(v) + "\n and my ranks is " + str(comm.rank))
#do some work on each process and then gather back onto root
v = v * v
recvbuf = comm.gather(v, root=0)
if comm.rank == 0:
        print("New array on rank 0:\n" + str(numpy.array(recvbuf)))

In [ ]:
!mpirun -n 2 --mca mpi_cuda_support 0 python gather.py

## Reduction
Create an array, scatter it, and do a parallel reduction.

In [ ]:
%%file reduce.py
from mpi4py import MPI
import numpy
comm = MPI.COMM_WORLD
sendbuf = []

if comm.rank == 0:
    m = numpy.array(range(comm.size * comm.size), dtype=float)
    m.shape=(comm.size, comm.size)
    print("Original array on root process\n" + str(m))
    sendbuf = m
    
# first scatter like before
v = comm.scatter(sendbuf, root=0)
print("I got this data:\n" + str(v) + "\n and my ranks is " + str(comm.rank))

recvbuf = comm.reduce(v, root=0)
if comm.rank == 0:
        print("New array on rank 0:\n" + str(numpy.array(recvbuf)))

In [ ]:
!mpirun -n 2 --mca mpi_cuda_support 0 python reduce.py

In [ ]:
%%file scatter_uppercase.py
<TO_DO>
import numpy as np

comm = <TO_DO>
rank = <TO_DO>
size = <TO_DO>

A = np.zeros((size,size))
if rank==0:
    A = np.random.randn(size, size)
    print("Original array on root process\n", A)
local_a = np.zeros(size)

comm.<TO_DO>(A, local_a, root=0)
print("Process", rank, "received", local_a)

In [ ]:
!mpirun -n 4 python scatter_uppercase.py